<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



<h1>Curso Ingeniería de Características</h1>

<h3>Visualización con componentes principales</h3>



<p> Julio Waissman Vilanova </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/reduccion-caracteristicas/pca.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;"  width="30" /> Ejecuta en Colab</a>

</center>

En esta libreta vamos a hacer una pequeña aplicación del Análisis en Componentes Principales (PCA) aplicado a los datos de marginación del estado de Sonora, los cuales los vamos a consultar [desde la página del gobierno federal para el año 2020](https://www.gob.mx/conapo/documentos/indices-de-marginacion-2020-284372).

No vamos a utilizar los indices normalizados de marginación por municipio desarrollados por la CONAPO ya que lo que queremos es hacer un análisis exploratorio de datos multivariados utilizndo componentes principales.

### 1. Cargando y seleccionando los datos de marginación

Empecemos por cargar los datos y ver como son:

In [ ]:
import pandas as pd

url = "http://www.conapo.gob.mx/work/models/CONAPO/Marginacion/Datos_Abiertos/Municipio/IMM_2020.xls"

dic_df = pd.read_excel(url, None)

dic_df.keys()

Nos vamos a quedar con sólo los municipios de sonora. En `IMM_2020` se encuentran los datos para fácil consumo de un programa, y en 'Diccionario' se encuentra la descripción de cada variable.

Vamos a ver en el diccionario de datos cuales son los datos que nos interesan.

In [ ]:
dic_df['Diccionario']

Vamos primero a separar los datos de sonora y revisar si se encuentran todos losmunicipios del estado:

In [ ]:
sonora = dic_df['IMM_2020'][dic_df['IMM_2020'].NOM_ENT.str.contains('Sonora')]
print(f"Los municipios de Sonora son: \n {sonora.NOM_MUN.unique()}")
sonora

Como podemos ver tenemos los 72 municipio. Nos vamos entonces a quedar con los datos de las columnas 5 a 14 (porcentajes) sin tomar en cuenta el tamaño de la población (en forma conciente)

In [ ]:
datos_municipio = sonora.iloc[:, 5:14]

# Vamos a ver, deberíamos tener 72 municipios, cada uno representado por 8 indicadores
print("Dimensión de los datos: ", datos_municipio.shape)

# Y los indicadores que vamos a usar
print(f"Las variables a usar son: \n{datos_municipio.head()}")

### 2. Aplicando el Análisis en componentes principales

Vamos a ver si podemos utilizar PCA para revisar estos datos que se encuentran en 9 dimensiones y ver si los datos de marginación tendrían un significado diferente si se analizan a nivel estatal. 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

datos = datos_municipio.values

n_components = 4
pipe = Pipeline(
    [('scaler', StandardScaler()), 
     ('pca', PCA(n_components=n_components))]
)

pipe.fit(datos)

pca = pipe.get_params()['pca']

pd.DataFrame(
    pca.components_.T, 
    columns=[f'PC {x}' for x in range(1,n_components+1)],
    index = datos_municipio.columns
)

¿Y cuanto explica sobre la varianza total cada una de las componentes? 

In [ ]:
import numpy as np
import plotly.express as px

print("Varianza por componente: ", pca.explained_variance_)
print("Porcentaje de varianza explicada por componente: ", 
      pca.explained_variance_ratio_)

df_ev = pd.DataFrame({
    "Varianza explicada": np.cumsum(pca.explained_variance_ratio_),
    "Componente": [f'PC {x}' for x in range(1,n_components+1)]
})
fig = px.line(df_ev, x='Componente', y='Varianza explicada', markers=True,
              title="Varianza explicada por componente")
fig.show()

Como vemos, menos del 47% de la varianza se explica por la componente principal, y menos del 69% con las dos primeras componentes. Por esta razón podemos decir que en Sonora, la marginación es un problema multidimensional.

Esto implica que atacar a una de las variables, no va a reducir a las otras variables, si no que es necesario desarrollar polñíticas mñas integrales. 

A nivel país esto no es el caso, donde la primer componente explica la mayor parte de la variación. ¿Porqué crees que eso pase?

In [ ]:
fig = px.bar(sonora.sort_values(by='PL.5000'), 
             x='NOM_MUN', y='PL.5000', 
             color="PL.5000",
             color_continuous_scale='bluered')
fig.show()

Y como lo vemos, pues la mayoría de los municipios son principalmente rurales. Separemos ahora los municipios con un `PL.5000` menor a 90 y revisemos:

In [ ]:
sonora_urbano = sonora[sonora['PL.5000'] < 90]
datos_urbano = sonora_urbano.iloc[:, 5:14]

datos = datos_urbano.values

n_components = 4
pipe_urbano = Pipeline(
    [('scaler', StandardScaler()), 
     ('pca', PCA(n_components=n_components))]
)

pipe_urbano.fit(datos)

pca_urbano = pipe_urbano.get_params()['pca']

pd.DataFrame(
    pca_urbano.components_.T, 
    columns=[f'PC {x}' for x in range(1,n_components+1)],
    index = datos_urbano.columns
)

In [ ]:
print("Varianza por componente: ", pca_urbano.explained_variance_)
print("Porcentaje de varianza explicada por componente: ", 
      pca_urbano.explained_variance_ratio_)

df_ev = pd.DataFrame({
    "Varianza explicada": np.cumsum(pca_urbano.explained_variance_ratio_),
    "Componente": [f'PC {x}' for x in range(1,n_components+1)]
})
fig = px.line(df_ev, x='Componente', y='Varianza explicada', markers=True,
              title="Varianza explicada por componente")
fig.show()

Y ahora veamos los municipios que llamaremos *rurales*

In [ ]:
sonora_rural = sonora[sonora['PL.5000'] > 90]
datos_rural = sonora_rural.iloc[:, 5:14]

datos = datos_rural.values

n_components = 4
pipe_rural = Pipeline(
    [('scaler', StandardScaler()), 
     ('pca', PCA(n_components=n_components))]
)

pipe_rural.fit(datos)

pca_rural = pipe_rural.get_params()['pca']

pd.DataFrame(
    pca_rural.components_.T, 
    columns=[f'PC {x}' for x in range(1,n_components+1)],
    index = datos_rural.columns
)

In [ ]:
print("Varianza por componente: ", pca_rural.explained_variance_)
print("Porcentaje de varianza explicada por componente: ", 
      pca_rural.explained_variance_ratio_)

df_ev = pd.DataFrame({
    "Varianza explicada": np.cumsum(pca_rural.explained_variance_ratio_),
    "Componente": [f'PC {x}' for x in range(1,n_components+1)]
})
fig = px.line(df_ev, x='Componente', y='Varianza explicada', markers=True,
              title="Varianza explicada por componente")
fig.show()

### 3. Visualización de los componentes principales

Vamos a ver si hay similaridades entre municipios utilizando los 4 componentes principales. Para esto vamos a utilizar las gráficas de `scatter` que nos permite visualizar la relación de hasta 4 variables diferentes.

Vamos a hacer el ejercicio con los datos de `sonora_urbano` y dejamos el análisis de los municipios rurales como ejercicio.

In [ ]:
df_urbano = pd.DataFrame(
    pipe_urbano.transform(datos_urbano.values),
    columns= [f'PC {x}' for x in range(1,n_components+1)],
    index= sonora_urbano.NOM_MUN    
)

fig = px.scatter(
    df_urbano, 
    x='PC 1', y='PC 2', color='PC 4', 
    text=df_urbano.index,
    title="Componentes principales de datos de marginación para municipios rurales de Sonora"
)
fig.show()